In [ ]:
import numpy as np
import pandas as pd
import fastf1
from src.utils import compute_track_dominance_multi
from src.plotset import setup_plot, save_fig, plot_track_dominance

from fastf1 import plotting
import matplotlib.pyplot as plt

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025,17,'Q')
session.load()

In [ ]:
points, segments, colors = compute_track_dominance_multi(session=session,drivers=['VER','SAI'],circuit_length=6003,window_size=600)

In [ ]:
fig = plot_track_dominance(points,segments,colors, figsize=(12,8))

In [ ]:
save_fig(fig=fig,name='track_dominance',loc='Reel21')

In [ ]:
corners = session.get_circuit_info().corners
rotation = session.get_circuit_info().rotation

In [ ]:
corners.loc[19,'Distance'] = 4800

In [ ]:
ref_lap = session.laps.pick_drivers('VER').pick_fastest().get_telemetry().copy()
comp_lap = session.laps.pick_drivers('SAI').pick_fastest().get_telemetry().copy()

In [ ]:
mult = ref_lap.Distance.iloc[-1]/comp_lap.Distance.iloc[-1]

In [ ]:
ref_dist = ref_lap.Distance.to_numpy()
ref_time = ref_lap.Time.dt.total_seconds().to_numpy()
comp_dist = (comp_lap.Distance * mult).to_numpy()
comp_time = comp_lap.Time.dt.total_seconds().to_numpy()

In [ ]:
comp_time = np.interp(ref_dist, comp_dist, comp_time)

In [ ]:
delta = comp_time - ref_time

In [ ]:
setup_plot(axeslabel=24, figtitle=28, legendfont=22)

fig, ax = plt.subplots(2,1,figsize=(15,12),height_ratios=[0.7,0.3],sharex=True)
ax[0].plot(ref_lap['Distance'],ref_lap['Speed'],color="#0004FF",linewidth=3, label='VERSTAPPEN')
ax[0].plot(comp_lap['Distance'],comp_lap['Speed'],color="#008CFF",linewidth=3, label='SAINZ')

ax[0].vlines(x=corners['Distance'], ymin=0, ymax=380,
          linestyles='dotted', colors='grey')
for _, corner in corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    ax[0].text(corner['Distance'], 390, txt,
            va='center_baseline', ha='center', size=12, color="#00FF0D")

# ax[0].set_title('Speed Trace', pad=30)
ax[0].set_xlim(ref_dist[0],ref_dist[-1])
ax[0].set_ylim(0, 400)
ax[0].set_ylabel('Speed (km/h)')
ax[0].legend()
ax[0].grid(visible=False)
ax02 = ax[0].twinx()
ax02.set_yticks([])
ax02.set_ylabel('BK',color="#292929",labelpad=30)

ax[1].plot(ref_lap['Distance'],delta,ls='--',lw=3,color='w')
ax[1].axhline(y=0,lw=1,ls='--',color="#FFFFFF")

ax[1].vlines(x=corners['Distance'], ymin=-0.6, ymax=0.6,
          linestyles='dotted', colors='grey')

ax[1].fill_between(x=(ref_dist[0],ref_dist[-1]),y1=0,y2=0.6,color='#0004FF',alpha=0.2)
ax[1].fill_between(x=(ref_dist[0],ref_dist[-1]),y1=-0.6,y2=0,color='#008CFF',alpha=0.2)

# ax[1].set_title('Lap Time Delta')
ax[1].set_xlim(ref_dist[0],ref_dist[-1])
ax[1].set_ylim(-0.6,0.6)
ax[1].set_ylabel('Delta (sec)')
ax[1].set_xlabel('Lap Distance (m)')
ax[1].grid(visible=False)

fig.align_labels(axs=ax)

In [ ]:
save_fig(fig=fig,name='telemetry',loc='Reel21')

In [ ]:
setup_plot(axeslabel=24, figtitle=28, legendfont=22)

fig, ax = plt.subplots(figsize=(15,8))

ax.vlines(x=corners['Distance'], ymin=-0.6, ymax=0.55,
          linestyles='dotted', colors='grey')

ax.plot(ref_lap['Distance'],delta,ls='--',lw=3,color='w')
ax.axhline(y=0,lw=1,ls='--',color="#FFFFFF")

ax.fill_between(x=(ref_dist[0],ref_dist[-1]),y1=0,y2=0.6,color='#0004FF',alpha=0.2)
ax.fill_between(x=(ref_dist[0],ref_dist[-1]),y1=-0.6,y2=0,color='#008CFF',alpha=0.2)

# ax.set_title('Lap Time Delta')
ax.set_xlim(ref_dist[0],ref_dist[-1])
ax.set_ylim(-0.6,0.6)
ax.set_ylabel('Delta (sec)')
ax.set_xlabel('Lap Distance (m)')
ax.grid(visible=False)

fig.align_labels(axs=ax)

In [ ]:
save_fig(fig=fig,name='delta',loc='Reel21')